 # Software Project :

- conda create -n software python=3.11.5
- conda activate software
- pip install ipykernel #VS code kernel library
- pip install pandas

In [ ]:
import pandas as pd

import sys
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../datasets')

import tdqm
import warnings

warnings.filterwarnings('ignore')